## Setting up our Python environment

In addition to the normal inputs, we'll grab Bessel functions from `scipy.special`. There are different conventions, but we'll mostly be using the $J_n$'s

For the rest of this notebook, I'll be assuming you reviewed bessel functions in the cylinder notebook.

In [1]:
import numpy as np, scipy as sp, matplotlib.pyplot as plt, seaborn as sns
sns.set_style({'axes.grid' : False})

from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact, fixed
from matplotlib import cm
from numpy import sin, cos, arctan, arctan2, array, sqrt, linspace, meshgrid, pi
#%matplotlib inline
import scipy.special
from scipy.special import j0,j1,jn,y0,y1,yn, jn_zeros

# Now on to drum heads

Assuming you solved this in class, you know we're looking for things like

$J_n(k_{m,n})R\cos(n\theta)\cos(k_{m,n}vt)$

So, let's plot that. Remember, we want the above function over the drum head, and zero elsewhere.

First, let's all read through Boas together

... reading ...

Now let's plot some things!

In [2]:
v = 1
periods = 2
frames_per = 50
Npoints = 100

x = np.linspace(-1,1,Npoints)
y = np.linspace(-1,1,Npoints)
X,Y = np.meshgrid(x,y)

def k(m,n):
    # jn_zeros(n, nt): Compute nt zeros of the Bessel function Jn(x).
    return jn_zeros(n,m)[m-1] # m is 0-indexed here

def generate(X, Y, t, n, m, v, f1, f2):
    theta = arctan2(Y,X) # This does arctan(Y/X) but gets the sign right.
    R = sqrt(X**2 + Y**2)
    # We know z = J_n(k*r)*cos(n*theta)*cos(k*v*t)
    # 
    f1 = {'sin':sin,'cos':cos}[f1]
    f2 = {'sin':sin,'cos':cos}[f2]

    result = jn(n,k(m,n)*R)*f1(n*theta)*f2(k(m,n)*v*t)
    result[R>1] = 0  # we plot points from the square, but physically require this.
    return result


def plot_frame(n=0,m=1,t=0,
              f1='cos',f2='cos',
              elev=20,azim=60,
              alpha=0.9):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    Z = generate(X,Y,t=t,n=n,m=m,v=v,f1=f1,f2=f2)
    #ax.plot_surface(X, Y, Z, rstride=4, cstride=4, alpha=0.3, cmap=cm.viridis, vmin=-1, vmax=1)
    ax.plot_surface(X, Y, Z, cmap=cm.viridis, vmin=-1, vmax=1, alpha=alpha)
    if n == 0:
        ax.set_zlim(-1,1)
    else:
        ax.set_zlim(-0.5,0.5)
    ax.view_init(elev=elev, azim=azim)
    plt.show()
_ = interact(plot_frame,t=(0,periods,periods/frames_per),
             n=(0,10),m=(1,10),
             f1=['sin','cos'],f2=['sin','cos'],
             elev=(-180,180,1),azim=(-180,180,1),
             alpha=(0,1,0.1),
            )

interactive(children=(IntSlider(value=0, description='n', max=10), IntSlider(value=1, description='m', max=10, min=1), FloatSlider(value=0.0, description='t', max=2.0, step=0.04), Dropdown(description='f1', index=1, options=('sin', 'cos'), value='cos'), Dropdown(description='f2', index=1, options=('sin', 'cos'), value='cos'), IntSlider(value=20, description='elev', max=180, min=-180), IntSlider(value=60, description='azim', max=180, min=-180), FloatSlider(value=0.9, description='alpha', max=1.0), Output()), _dom_classes=('widget-interact',))

And now let's plot a grid, complete with nodelines!

In [3]:
v = 1
periods = 2
frames_per = 50
Npoints = 100

x = np.linspace(-1,1,Npoints)
y = np.linspace(-1,1,Npoints)
X,Y = np.meshgrid(x,y)


def plot_frame2(t=0,nmax=1,mmax=2):
    # one period is 2*pi/jn_zeros(n,m)[m-1]
    #t = t*2*pi/jn_zeros(n,m)[m-1]
    ns = list(range(0,nmax))
    ms = list(range(1,mmax))
    fig = plt.figure(figsize=(6*len(ms),2*(len(ns))))
    
    axs = {}
    rows, cols = len(ns), 2*len(ms)

    idx = 1
    for m in ms:
        axs[m] = {}
        for n in ns:
            axs[m][n] = (fig.add_subplot(rows,cols,idx, projection='3d'),
                        fig.add_subplot(rows,cols,idx+1))
            idx += 2

    
    for m in ms:
        for n in ns:
            Z = generate(X, Y, t, n, m, v, 'cos', 'cos')
            axs[m][n][0].plot_surface(X, Y, Z, alpha=0.9, 
                                      cmap=cm.viridis, vmin=-0.7,vmax=0.7,)

            if n == 0:
                axs[m][n][0].set_zlim(-0.7,.7)
                axs[m][n][1].imshow(Z,vmin=-0.7,vmax=0.7, cmap=cm.viridis)
            else:
                axs[m][n][0].set_zlim(-0.5,0.5)
                axs[m][n][1].imshow(Z,vmin=-0.5,vmax=0.5, cmap=cm.viridis)
            # The funny business with levels here is because you won't
            # get a contour exactly at zero that necessarily tracks
            # around both sides of the circle due to the fact that
            # we've discretized things.
            levels = [-0.000000001,0.0,0.000000001]
            axs[m][n][1].contour(Z, levels, colors='k',
                                 linestyles='solid', linewidths=2)
    plt.show()
_ = interact(plot_frame2,t=(0,periods,periods/frames_per),
             nmax=(1,5),mmax=(2,5),
            )

interactive(children=(FloatSlider(value=0.0, description='t', max=2.0, step=0.04), IntSlider(value=1, description='nmax', max=5, min=1), IntSlider(value=2, description='mmax', max=5, min=2), Output()), _dom_classes=('widget-interact',))